# Tutorial Básico de Criptografía Homomórfica

En este tutorial básico vamos a ver como se pueden cifrar datos con esquemas de criptografía homomórfica, realizar operaciones y comprobar que el resultado es igual que si las operaciones se hubiesen hecho con los datos sin cifrar.

In [1]:
# En primer lugar instalamos la libreria de Microsfot TenSEAL que nos permite realizar las operaciones
! pip install tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 32.2 MB/s eta 0:00:00


# Elementos Auxiliares

Para este tutorial vamos a necesitar importar las librearías de TenSEAL y random (generación de números aleatorios). Además nos definimos dos funciones que serán útiles más adelante en el tutorial.

In [2]:
# Importamos las librerías necesarias
import tenseal as ts
import random

# Método que nos genera un vector de tamaño size con números aleatorios hasta el 4
def generate_vector(size):
    return [random.randint(0, 4) for _ in range(size)]

# Nos calcula la diferencia media entre elementos de una lista
def average_difference_between_lists(list1, list2):
    if len(list1) != len(list2):
        raise ValueError("Lists must be of the same size.")

    differences = [abs(list1[i] - list2[i]) for i in range(len(list1))]
    avg_diff = sum(differences) / len(differences)
    return avg_diff

# Contextos

Uno de los objetos más importantes en TenSEAL es el contexto. Este objeto guarda información relevante como las claves de cifrado y los parámetros del esquema de cifrado homórfico utilizado. De esta manera sólo hará falta utilizar este objeto para la mayoría de operaciones necesarias. En este caso vamos a crear un contexto para empezar a realizar nuestras operaciones.

In [3]:
# Creación del contexto

# Controla la precisión de la parte decimal
bits_scale = 26

# Creación del contexto en TenSEAL. Utilizamos CKKS que permite realizar cómputos con números reales
# CKKS fue presentado en ASCIACCS 17: https://eprint.iacr.org/2016/421.pdf
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[31, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, 31]
)
# Parámetros adicionales
context.global_scale = pow(2, bits_scale)
context.generate_galois_keys()

# Cifrando un Vector

El siguiente código cifra un vector simple

In [4]:
plain_vector = [60, 66, 73, 81, 90]
encrypted_vector = ts.ckks_vector(context, plain_vector)
print("Tamaño del vector cifrado:", encrypted_vector.size())
encrypted_vector

Tamaño del vector cifrado: 5


Ahora vamos a proceder a realizar una suma

In [5]:
add_result = encrypted_vector + [1, 2, 3, 4, 5]
print(add_result.decrypt())

[61.000001106768615, 67.99999118584887, 75.99998668182143, 84.99998778581144, 95.00000134110331]


¿Qué valores obtenemos?

Con la multiplicación pasa algo similar

In [6]:
mul_result = encrypted_vector * [1, 2, 3, 4, 5]
print(mul_result.decrypt())

[60.05864551683268, 132.1290271253361, 219.21405784287478, 324.316668144372, 450.4398756057921]


Ahora vamos a realizar operaciones y observar como el error se va acumulando hasta que llega a un límite definido por nosotros. El código proporcionado muestra como el 'error' se va acumulando según se siguen realizando operaciones con los datos. Los algoritmos de criptografía homomórfica tienen mecanismos para ir reduciendo este 'error' pero hacen que la computación sea muy ineficiente comparada con los cómputos sobre datos sin cifrar.

In [9]:
# Generamos un vector aleatorio
vector_size = 5
original_vector = generate_vector(vector_size)
print("Vector Original:", original_vector)
# Ciframos el vector
encrypted_vector = ts.ckks_vector(context, original_vector)
decrypted_result = []
original_result = original_vector
max_avg_error = 10
# Realizamos operaciones hasta llegar a un error máximo
for i in range(1000):
    # Multiplicamos el vector cifrado
    encrypted_vector = encrypted_vector * [1, 2, 3, 4, 5]
    # Multiplicamos el vector original
    original_result = [a * b for a, b in zip(original_result, [1, 2, 3, 4, 5])]
    # Desciframos el resultado y mostramos
    decrypted_result = encrypted_vector.decrypt()
    print("Resultado Descifrado:", decrypted_result)
    print("Resultado Original:", original_result)
    # Si la diferencia supera un límite salimos
    print("Diferencia media: ", average_difference_between_lists(original_result,decrypted_result))
    if average_difference_between_lists(original_result,decrypted_result)>max_avg_error:
        print("Salimos en la iteración", i)
        break

Vector Original: [1, 2, 2, 0, 4]
Resultado Descifrado: [1.0009748379707792, 4.003888247070249, 6.005863077849119, 8.153625758036323e-05, 20.019568224918785]
Resultado Original: [1, 4, 6, 0, 20]
Diferencia media:  0.006075184813302497
Resultado Descifrado: [1.0026804732167645, 8.021465237565975, 18.048422695704378, 0.0003100528937171101, 100.26918374616335]
Resultado Original: [1, 8, 18, 0, 100]
Diferencia media:  0.06841244110883686
Resultado Descifrado: [1.0048788129495527, 16.07826651100001, 54.26450823065771, 0.0012294838366049051, 502.4499464402027]
Resultado Original: [1, 16, 54, 0, 500]
Diferencia media:  0.5597658957293202
Resultado Descifrado: [1.0093092223151294, 32.29846069612703, 163.51210466994326, 0.004937573698783485, 2523.3384756036444]
Resultado Original: [1, 32, 162, 0, 2500]
Diferencia media:  5.032657553145716
Resultado Descifrado: [1.0177539545253222, 65.13762845401527, 494.64231125407247, 0.019908498342506895, 12722.296613548006]
Resultado Original: [1, 64, 486, 0,

# Tarea: Puedes resolver el problema de los millonarios con criptografía homomórfica?

cifrar y hacer la resta